In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
# Load data
data_23 = pd.read_csv("data/0901_Tek023.csv")
data_interp = pd.read_csv("data/0901_Tek023_interpolated.csv")

print("Original data shape:", data_23.shape)
print("Interpolated data shape:", data_interp.shape)
data_23.head()

In [ ]:
# Constants and parameters
epsilon = 1e-9

# Prepare data
# Use interpolated data for training
train_mask = data_interp["t"] <= 1000
test_mask = data_interp["t"] > 1000

t_train = data_interp["t"][train_mask].values
log_t_train = data_interp["log_t"][train_mask].values
vth_train = data_interp["vth_average"][train_mask].values

t_test = data_interp["t"][test_mask].values
log_t_test = data_interp["log_t"][test_mask].values
vth_test = data_interp["vth_average"][test_mask].values

print(f"Training samples: {len(t_train)}")
print(f"Test samples: {len(t_test)}")

In [ ]:
# Scaling
scaler_t = MinMaxScaler()
scaler_vth = MinMaxScaler()

log_t_train_scaled = scaler_t.fit_transform(log_t_train.reshape(-1, 1))
log_t_test_scaled = scaler_t.transform(log_t_test.reshape(-1, 1))

vth_train_scaled = scaler_vth.fit_transform(vth_train.reshape(-1, 1))
vth_test_scaled = scaler_vth.transform(vth_test.reshape(-1, 1))

# Convert to tensors
t_train_tf = tf.constant(log_t_train_scaled, dtype=tf.float32)
t_test_tf = tf.constant(log_t_test_scaled, dtype=tf.float32)
vth_train_tf = tf.constant(vth_train_scaled, dtype=tf.float32)
vth_test_tf = tf.constant(vth_test_scaled, dtype=tf.float32)

## Physics Loss Function

**TODO**: Define the physics-based loss function based on GaN device degradation mechanisms.

Possible physics to incorporate:
- Trap-assisted degradation: $\frac{dV_{th}}{dt} = f(V_{th}, t, T, ...)$
- Power-law degradation: $V_{th}(t) = V_{th,0} + A \cdot t^n$
- Logarithmic degradation: $V_{th}(t) = V_{th,0} + A \cdot \ln(1 + t/\tau)$
- Arrhenius temperature dependence
- Interface trap dynamics

The physics loss can enforce:
1. Governing differential equations
2. Boundary/initial conditions
3. Physical constraints (e.g., monotonicity, bounds)

In [ ]:
# =============================================================================
# PHYSICS LOSS FUNCTION - TO BE MODIFIED
# =============================================================================

def physics_loss(model, t_input):
    """
    Compute the physics-based loss term.
    
    This function should encode the physical laws governing Vth degradation.
    Modify this function to incorporate the correct physics.
    
    Parameters:
    -----------
    model : keras.Model
        The neural network model
    t_input : tf.Tensor
        Time input tensor (scaled)
    
    Returns:
    --------
    tf.Tensor
        Physics loss value
    """
    
    with tf.GradientTape() as tape:
        tape.watch(t_input)
        vth_pred = model(t_input, training=True)
    
    # Compute dVth/dt (gradient of Vth with respect to time)
    dvth_dt = tape.gradient(vth_pred, t_input)
    
    # =============================================================================
    # PLACEHOLDER PHYSICS EQUATION - MODIFY THIS
    # =============================================================================
    # Example: Enforce that dVth/dt follows some physical law
    # 
    # Option 1: Simple power-law constraint (dVth/dt ~ t^(n-1))
    # Option 2: Logarithmic constraint (dVth/dt ~ 1/t)
    # Option 3: Custom differential equation
    #
    # Current placeholder: Just penalize large derivatives (smoothness)
    # Replace with actual physics equation when ready
    
    # Placeholder: smoothness regularization
    physics_residual = dvth_dt  # Modify this to be the actual physics residual
    
    # Example physics residual for power-law: dVth/dt = A * n * t^(n-1)
    # A = 0.01  # amplitude (to be fitted or set)
    # n = 0.2   # exponent (to be fitted or set)
    # expected_dvth_dt = A * n * tf.pow(t_input + epsilon, n - 1)
    # physics_residual = dvth_dt - expected_dvth_dt
    
    # Return mean squared physics residual
    return tf.reduce_mean(tf.square(physics_residual))

In [ ]:
# =============================================================================
# ADDITIONAL PHYSICS CONSTRAINTS - TO BE MODIFIED
# =============================================================================

def boundary_condition_loss(model, t_initial, vth_initial):
    """
    Enforce initial/boundary conditions.
    
    Parameters:
    -----------
    model : keras.Model
        The neural network model
    t_initial : tf.Tensor
        Initial time point(s)
    vth_initial : tf.Tensor
        Known Vth at initial time
    
    Returns:
    --------
    tf.Tensor
        Boundary condition loss
    """
    vth_pred_initial = model(t_initial, training=True)
    return tf.reduce_mean(tf.square(vth_pred_initial - vth_initial))


def monotonicity_loss(model, t_input):
    """
    Enforce monotonic increase/decrease of Vth over time (if physically expected).
    
    Parameters:
    -----------
    model : keras.Model
        The neural network model
    t_input : tf.Tensor
        Time input tensor
    
    Returns:
    --------
    tf.Tensor
        Monotonicity loss (penalizes negative derivatives if expecting increase)
    """
    with tf.GradientTape() as tape:
        tape.watch(t_input)
        vth_pred = model(t_input, training=True)
    
    dvth_dt = tape.gradient(vth_pred, t_input)
    
    # Penalize negative derivatives (if Vth should increase)
    # Use ReLU to only penalize negative values
    violation = tf.nn.relu(-dvth_dt)  # Penalize if dvth_dt < 0
    
    return tf.reduce_mean(tf.square(violation))

In [ ]:
# =============================================================================
# PINN MODEL DEFINITION
# =============================================================================

def create_pinn_model(input_dim=1, hidden_layers=[64, 64, 32], output_dim=1):
    """
    Create a neural network for PINN.
    
    Parameters:
    -----------
    input_dim : int
        Number of input features (typically 1 for time)
    hidden_layers : list
        List of hidden layer sizes
    output_dim : int
        Number of outputs (typically 1 for Vth)
    
    Returns:
    --------
    keras.Model
        The PINN model
    """
    inputs = layers.Input(shape=(input_dim,))
    x = inputs
    
    for units in hidden_layers:
        x = layers.Dense(units, activation='tanh')(x)  # tanh is common for PINNs
    
    outputs = layers.Dense(output_dim)(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Create model
pinn_model = create_pinn_model(input_dim=1, hidden_layers=[64, 64, 32], output_dim=1)
pinn_model.summary()

In [ ]:
# =============================================================================
# LOSS WEIGHTS - ADJUST THESE
# =============================================================================

# Weight for data loss (fitting to measured data)
lambda_data = 1.0

# Weight for physics loss (enforcing physical equations)
lambda_physics = 0.1  # Start small, increase if physics is important

# Weight for boundary condition loss
lambda_bc = 0.1

# Weight for monotonicity loss (optional)
lambda_mono = 0.0  # Set > 0 if Vth should be monotonic

In [ ]:
# =============================================================================
# CUSTOM TRAINING LOOP
# =============================================================================

optimizer = keras.optimizers.Adam(learning_rate=0.001)

@tf.function
def train_step(t_data, vth_data, t_collocation):
    """
    Single training step for PINN.
    
    Parameters:
    -----------
    t_data : tf.Tensor
        Time points with known Vth values
    vth_data : tf.Tensor
        Known Vth values at t_data
    t_collocation : tf.Tensor
        Collocation points for physics loss (can be same as t_data or denser)
    
    Returns:
    --------
    dict
        Dictionary of loss values
    """
    with tf.GradientTape() as tape:
        # Data loss
        vth_pred = pinn_model(t_data, training=True)
        data_loss = tf.reduce_mean(tf.square(vth_pred - vth_data))
        
        # Physics loss
        phys_loss = physics_loss(pinn_model, t_collocation)
        
        # Boundary condition loss (use first point as initial condition)
        bc_loss = boundary_condition_loss(
            pinn_model, 
            t_data[:1], 
            vth_data[:1]
        )
        
        # Monotonicity loss (optional)
        mono_loss = monotonicity_loss(pinn_model, t_collocation)
        
        # Total loss
        total_loss = (lambda_data * data_loss + 
                      lambda_physics * phys_loss + 
                      lambda_bc * bc_loss +
                      lambda_mono * mono_loss)
    
    # Compute gradients and update weights
    gradients = tape.gradient(total_loss, pinn_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, pinn_model.trainable_variables))
    
    return {
        'total_loss': total_loss,
        'data_loss': data_loss,
        'physics_loss': phys_loss,
        'bc_loss': bc_loss,
        'mono_loss': mono_loss
    }

In [ ]:
# =============================================================================
# TRAINING
# =============================================================================

epochs = 5000
print_every = 500

# Create collocation points (can be denser than data points)
# Using training data points as collocation points
t_collocation = t_train_tf

# Store loss history
loss_history = {
    'total_loss': [],
    'data_loss': [],
    'physics_loss': [],
    'bc_loss': [],
    'mono_loss': []
}

print("Starting PINN training...")
print(f"Loss weights: data={lambda_data}, physics={lambda_physics}, bc={lambda_bc}, mono={lambda_mono}")
print("-" * 80)

for epoch in range(epochs):
    losses = train_step(t_train_tf, vth_train_tf, t_collocation)
    
    # Store losses
    for key in loss_history:
        loss_history[key].append(losses[key].numpy())
    
    if (epoch + 1) % print_every == 0:
        print(f"Epoch {epoch+1}/{epochs}:")
        print(f"  Total: {losses['total_loss']:.6f}, Data: {losses['data_loss']:.6f}, "
              f"Physics: {losses['physics_loss']:.6f}, BC: {losses['bc_loss']:.6f}")

print("-" * 80)
print("Training complete!")

In [ ]:
# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].semilogy(loss_history['total_loss'])
axes[0, 0].set_title('Total Loss')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')

axes[0, 1].semilogy(loss_history['data_loss'])
axes[0, 1].set_title('Data Loss')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')

axes[1, 0].semilogy(loss_history['physics_loss'])
axes[1, 0].set_title('Physics Loss')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Loss')

axes[1, 1].semilogy(loss_history['bc_loss'])
axes[1, 1].set_title('Boundary Condition Loss')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Loss')

plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# EVALUATION
# =============================================================================

# Predict on training and test data
vth_pred_train_scaled = pinn_model.predict(t_train_tf)
vth_pred_test_scaled = pinn_model.predict(t_test_tf)

# Inverse transform to original scale
vth_pred_train = scaler_vth.inverse_transform(vth_pred_train_scaled).flatten()
vth_pred_test = scaler_vth.inverse_transform(vth_pred_test_scaled).flatten()

# Calculate metrics
def calculate_metrics(y_true, y_pred, name="Model"):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n{name}:")
    print(f"  MSE:  {mse:.6f}")
    print(f"  RMSE: {rmse:.6f}")
    print(f"  MAE:  {mae:.6f}")
    print(f"  R²:   {r2:.6f}")
    
    return {"MSE": mse, "RMSE": rmse, "MAE": mae, "R2": r2}

print("=== PINN Model Evaluation ===")
train_metrics = calculate_metrics(vth_train, vth_pred_train, "Training Set")
test_metrics = calculate_metrics(vth_test, vth_pred_test, "Test Set (t > 1000s)")

In [ ]:
# Plot predictions vs actual
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Training range
axes[0].plot(log_t_train, vth_train, 'k-', linewidth=2, label="Actual")
axes[0].plot(log_t_train, vth_pred_train, 'r--', alpha=0.8, label="PINN")
axes[0].set_xlabel("log(Time)")
axes[0].set_ylabel("Vth")
axes[0].set_title("Training Range (t <= 1000s)")
axes[0].legend()

# Test range
axes[1].plot(log_t_test, vth_test, 'k-', linewidth=2, label="Actual")
axes[1].plot(log_t_test, vth_pred_test, 'r--', alpha=0.8, label="PINN")
axes[1].set_xlabel("log(Time)")
axes[1].set_ylabel("Vth")
axes[1].set_title("Test Range (t > 1000s)")
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Full range plot
plt.figure(figsize=(12, 6))

# Combine all data
all_log_t = np.concatenate([log_t_train, log_t_test])
all_vth_actual = np.concatenate([vth_train, vth_test])
all_vth_pred = np.concatenate([vth_pred_train, vth_pred_test])

plt.plot(all_log_t, all_vth_actual, 'k-', linewidth=2, label="Actual")
plt.plot(all_log_t, all_vth_pred, 'r--', alpha=0.8, label="PINN Prediction")
plt.axvline(x=np.log(1000), color='gray', linestyle=':', label="Train/Test Split (t=1000s)")
plt.xlabel("log(Time)")
plt.ylabel("Vth")
plt.title("PINN: Full Range Prediction")
plt.legend()
plt.show()

In [ ]:
# =============================================================================
# PREDICT AT SPECIFIC TIME (10000s)
# =============================================================================

# Find 10000s in original data
idx_10000 = (np.abs(data_23["t"] - 10000)).idxmin()
actual_vth_10000 = data_23["vth_average"][idx_10000]
log_t_10000 = np.log(data_23["t"][idx_10000] + epsilon)

# Scale and predict
log_t_10000_scaled = scaler_t.transform([[log_t_10000]])
vth_pred_10000_scaled = pinn_model.predict(log_t_10000_scaled)
vth_pred_10000 = scaler_vth.inverse_transform(vth_pred_10000_scaled)[0, 0]

print(f"\n=== Prediction at t=10000s ===")
print(f"Actual Vth:    {actual_vth_10000:.6f}")
print(f"PINN Predicted: {vth_pred_10000:.6f}")
print(f"Error:         {abs(actual_vth_10000 - vth_pred_10000):.6f}")

## Next Steps

1. **Modify `physics_loss()` function** with the correct physical equations for GaN Vth degradation
2. **Tune loss weights** (`lambda_data`, `lambda_physics`, `lambda_bc`, `lambda_mono`)
3. **Add more physics constraints** if needed (temperature dependence, stress conditions, etc.)
4. **Compare with pure data-driven models** to see if physics helps extrapolation

In [ ]:
# =============================================================================
# TEMPLATE FOR CUSTOM PHYSICS LOSS
# =============================================================================

# Uncomment and modify the physics loss function below when ready

# def physics_loss_custom(model, t_input):
#     """
#     Custom physics loss based on GaN degradation mechanism.
#     
#     Example: Power-law degradation
#     Vth(t) = Vth0 + A * t^n
#     => dVth/dt = A * n * t^(n-1)
#     """
#     with tf.GradientTape() as tape:
#         tape.watch(t_input)
#         vth_pred = model(t_input, training=True)
#     
#     dvth_dt = tape.gradient(vth_pred, t_input)
#     
#     # Physical parameters (to be determined from experiments or fitting)
#     A = 0.01   # Amplitude coefficient
#     n = 0.2    # Power-law exponent
#     
#     # Expected derivative from physics
#     # Note: t_input is scaled, need to transform back or use scaled equation
#     expected_dvth_dt = A * n * tf.pow(t_input + epsilon, n - 1)
#     
#     # Physics residual
#     residual = dvth_dt - expected_dvth_dt
#     
#     return tf.reduce_mean(tf.square(residual))